# Counting number of the mutual friends
- For each user having ID in the column userId count the amount of his / her common friends with each other user having ID in the column userId.

- Print 49 pairs of the users having the largest amount of common friends, ordered in descending order first by the common friends count , then by id of user1 and finally by id of user 2. The format is following: "count user1 user2"7

In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

## Explore dataset

In [4]:
g = sparkSession.read.parquet(graphPath)
g.show(5)

+--------+--------------------+
|    user|             friends|
+--------+--------------------+
|22991438|[20699, 175973, 5...|
|37586597|[83616, 139192, 1...|
|56325000|[504270, 645333, ...|
|12862761|[234344, 5991561,...|
|38989299|[47992, 83113, 70...|
+--------+--------------------+
only showing top 5 rows



In [5]:
g.printSchema()

root
 |-- user: integer (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: integer (containsNull = true)



## List top 50 friends that have most common user

In [6]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number
from pyspark.sql import Window

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users_size", size("users")) \

In [7]:
reversedGraph.printSchema()

root
 |-- friend: integer (nullable = true)
 |-- users: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- users_size: integer (nullable = false)



In [8]:
reversedGraph.show(5)

+------+--------------------+----------+
|friend|               users|users_size|
+------+--------------------+----------+
|   148|[65051219, 146311...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
| 10362|          [65278216]|         1|
| 11458|          [39169321]|         1|
+------+--------------------+----------+
only showing top 5 rows



In [9]:
# Method 1
window = Window.orderBy(col("users_size").desc())

top50 = reversedGraph.withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .select(col("friend"), col("users_size")) \
            .orderBy(col("users_size").desc()) \
            .collect()
top50[0:5]

[Row(friend=9606655, users_size=244),
 Row(friend=62922315, users_size=241),
 Row(friend=1288836, users_size=240),
 Row(friend=36402159, users_size=239),
 Row(friend=36079654, users_size=239)]

In [10]:
# Method 2
reversedGraph \
    .select("friend", "users_size") \
    .orderBy(col("users_size").desc()) \
    .limit(5).show()

+--------+----------+
|  friend|users_size|
+--------+----------+
| 9606655|       244|
|62922315|       241|
| 1288836|       240|
|36079654|       239|
|36402159|       239|
+--------+----------+



## Counting number of the mutual friends

1-Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)<br \>
2-group the resulting dataframe by the column “friend” (like in the demo iPython notebook)<br \>
3-create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)

In [11]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number
from pyspark.sql import Window
import pyspark.sql.functions as f 

reversedGraph = sparkSession.read.parquet(graphPath) \
    .select(
        f.explode('friends').alias('friend'),
        'user'
    ) \
    .groupBy('friend') \
    .agg(collect_list('user').alias('users'))

reversedGraph.show(5)

+------+--------------------+
|friend|               users|
+------+--------------------+
|   148|[65051219, 146311...|
|  5518|          [58573511]|
|  9900|          [36844066]|
| 10362|          [65278216]|
| 11458|          [39169321]|
+------+--------------------+
only showing top 5 rows



4-sort the elements in the column “users” by the function sort_array<br \>
5-filter only the rows which have more than 1 element in the column “users”<br \>

In [12]:
df = reversedGraph \
    .select(
        'friend',
        f.sort_array('users').alias('users')
    ) \
    .where(size('users') > '1')

df.show(5)

+------+--------------------+
|friend|               users|
+------+--------------------+
|   148|[3195315, 1463110...|
| 36538|[5506394, 6170161...|
| 41751|[41811068, 60873111]|
| 49331|[45058971, 58571716]|
| 73470|[37445156, 49852791]|
+------+--------------------+
only showing top 5 rows



7-for each row emit all possible ordered pairs of users from the column “users” (tip: write a user defined function for this)<br \>
8-count the number of times each pair has appeared<br \>
9-with the help of the window function (like in the demo python notebook) select 49 pairs of users who have the biggest amount of common friends

In [13]:
from itertools import combinations
from pyspark.sql.functions import udf
from pyspark.sql import types

def emit_all_pairs(users):
    return list(combinations(users, 2))

schema = types.ArrayType(types.IntegerType())
udf_emit_all_pairs = udf(emit_all_pairs, types.ArrayType(schema))

In [14]:
result = df \
    .withColumn(
        'pairs', udf_emit_all_pairs(col('users'))) \
    .select(
        f.explode('pairs').alias('pair')) \
    .withColumn('a', col('pair').getItem(0)) \
    .withColumn('c', col('pair').getItem(1)) \
    .groupBy('a', 'c') \
    .agg(f.count('*').alias('number of the mutual friends')) \
    .orderBy(f.desc('number of the mutual friends'), f.desc('a'), f.desc('c'))

result.show(5)

+--------+--------+----------------------------+
|       a|       c|number of the mutual friends|
+--------+--------+----------------------------+
|27967558|42973992|                        3206|
|20158643|42973992|                        3130|
|22582764|42973992|                        3066|
|21864412|51640390|                        3044|
|17139850|51640390|                        3021|
+--------+--------+----------------------------+
only showing top 5 rows



In [15]:
for row in result.take(49):
    print row['number of the mutual friends'], row['a'], row['c']

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985